In [ ]:
!nvidia-smi

Tue Feb  2 12:20:18 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%%capture
!pip install transformers datasets

In [ ]:
import os
import random
import torch
import numpy as np

def seed_all(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_all(42)

In [ ]:
from datasets import load_dataset, load_metric

task = "cola"
dataset = load_dataset("glue", task)
metric = load_metric("glue", task)

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/cola/1.0.0/7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4. Subsequent calls will reuse this data.


In [ ]:
dataset.keys()

dict_keys(['train', 'validation', 'test'])

In [ ]:
dataset["train"][0]

{'idx': 0,
 'label': 1,
 'sentence': "Our friends won't buy this analysis, let alone the next one we propose."}

In [ ]:
dataset.pop("test")

Dataset({
    features: ['sentence', 'label', 'idx'],
    num_rows: 1063
})

In [ ]:
from transformers import TrainingArguments

batch_size = 16

args = TrainingArguments(
    "test-glue",
    evaluation_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="matthews_correlation",
)

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer

device = "cuda" if torch.cuda.is_available() else "cpu"

def experiment(model_name):
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name, num_labels=2
    ).to(torch.device(device))
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    encoded_dataset = dataset.map(lambda example: tokenizer(example['sentence'], truncation=True), batched=True)
    encoded_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"], device=device)
    trainer = Trainer(
        model,
        args,
        train_dataset=encoded_dataset["train"],
        eval_dataset=encoded_dataset["validation"],
        tokenizer=tokenizer, 
        compute_metrics=compute_metrics,
    )
    trainer.train()
    result = trainer.evaluate()["eval_matthews_correlation"]
    return (trainer.model, result)

In [ ]:
results = {}
model_names = ["bert-base-uncased", "distilbert-base-uncased", "distilroberta-base"]

for model_name in model_names:
    print(f"=====Tuning {model_name}=====")
    results[model_name] = experiment(model_name)

=====Tuning bert-base-uncased=====


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

/usr/local/lib/python3.6/dist-packages/datasets/arrow_dataset.py:851: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.tensor(x, **format_kwargs)


Epoch,Training Loss,Validation Loss,Matthews Correlation,Runtime,Samples Per Second
1,0.488000,0.464488,0.520845,2.460500,423.904000
2,0.291500,0.480295,0.534564,2.368500,440.368000
3,0.166600,0.810376,0.571744,2.330600,447.524000
4,0.104200,0.928752,0.555285,2.263800,460.737000
5,0.059900,1.130933,0.556995,2.285800,456.299000


=====Tuning distilbert-base-uncased=====


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

Epoch,Training Loss,Validation Loss,Matthews Correlation,Runtime,Samples Per Second
1,0.520200,0.480449,0.448542,1.846400,564.885000
2,0.334000,0.540248,0.453199,1.863100,559.806000
3,0.201700,0.739443,0.496715,1.818600,573.506000
4,0.130100,0.933062,0.506990,1.820200,573.019000
5,0.091800,1.077799,0.483239,1.836400,567.952000


=====Tuning distilroberta-base=====


Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.weight'

Epoch,Training Loss,Validation Loss,Matthews Correlation,Runtime,Samples Per Second
1,0.543100,0.567221,0.360621,1.844800,565.364000
2,0.390900,0.490205,0.466391,1.897600,549.648000
3,0.264800,0.565403,0.526933,1.820600,572.887000
4,0.194600,0.833024,0.551277,1.842700,566.018000
5,0.136600,0.977999,0.536768,1.892800,551.041000


In [ ]:
for model_name, (_, score) in results.items():
    print(f"{model_name}: {score:.5f}")

bert-base-uncased: 0.57174
distilbert-base-uncased: 0.50699
distilroberta-base: 0.55128


In [ ]:
best_model_name = max(results, key=lambda k: results[k][1])
best_model = results[best_model_name][0]
file_name = f"custom-{best_model_name}"
best_model.save_pretrained(file_name)

In [ ]:
!zip -r custom-bert-base-uncased.zip custom-bert-base-uncased/

  adding: custom-bert-base-uncased/ (stored 0%)
  adding: custom-bert-base-uncased/config.json (deflated 46%)
  adding: custom-bert-base-uncased/pytorch_model.bin (deflated 7%)


In [ ]:
from google.colab import files

files.download(f"{file_name}.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>